# Thai Medical QA / RAG — Dataset + Data Engineering Starter (Notebook)

This notebook uses **Thaweewat/thai-med-pack** from Hugging Face and helps you:

- Load + inspect the dataset (Thai medical Q&A style)
- Parse instruction-style text into structured fields: `question`, `answer`
- Do basic data engineering (cleaning, dedupe, privacy/PII-risk scan, safety filtering, split)
- Build a retrieval corpus by chunking answers/contexts
- Export `jsonl` files for model training / RAG indexing

> ⚠️ Note: Some entries may include **sensitive topics** (e.g., sexual health, minors, mental health crises).  
> For an academic MVP, you should **filter** sensitive content and keep the system as **decision-support only**.


In [3]:
# If needed, install deps (uncomment)
# !pip install -U datasets pandas

import re, json, os, random, hashlib
import pandas as pd
from datasets import load_dataset
import tqdm as notebook_tqdm

RANDOM_SEED = 42
random.seed(RANDOM_SEED)


## 1) Load dataset: Thaweewat/thai-med-pack

This dataset is provided as an instruction-style text field (typically one column: `text`).
We'll load the `train` split and parse the `[INST] ... [/INST]` part into question & answer.


In [4]:
ds = load_dataset("Thaweewat/thai-med-pack")
# Typically only 'train' split is provided
df_raw = ds[list(ds.keys())[0]].to_pandas()
df_raw.head()


Generating train split: 100%|██████████| 189190/189190 [00:00<00:00, 215524.59 examples/s]


,text
0,<s>[INST] สวัสดีค่ะ อยากรู้ว่าต้องทำยังไงคะมีอ...
1,<s>[INST] อยากทราบว่ากินยาคลุมฉุกเฉินชนิด 1 เม...
2,<s>[INST] ไม่รู้ว่าใช่ประจำเดือนหรือป่าว แต่เล...
3,<s>[INST] ตอนนี้หนูอายุ13ปีและหนูเป็นนางรำของร...
4,<s>[INST] ตามต้นหัวข้อเลยค่ะ เราเป็นคนที่คิดมา...


## 2) Parse instruction format into structured QA

Expected pattern in `text` (example):
- `<s>[INST] ...question... [/INST] ...answer... </s>`

We will extract:
- `question`
- `answer`

If an entry does not match the pattern, it will be dropped (or kept in `unparsed` for audit).


In [5]:
WS_RE = re.compile(r"\s+")
INST_RE = re.compile(r"\[INST\](.*?)\[/INST\](.*)", re.DOTALL)

def sha1(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8", errors="ignore")).hexdigest()

def clean_text(s) -> str:
    if s is None:
        return ""
    s = str(s).replace("\u00a0", " ")
    s = WS_RE.sub(" ", s).strip()
    return s

def parse_inst(sample: str):
    if not sample:
        return None, None
    m = INST_RE.search(sample)
    if not m:
        return None, None
    q = clean_text(m.group(1))
    a = clean_text(m.group(2))
    # strip common special tokens
    q = q.replace("<s>", "").replace("</s>", "").strip()
    a = a.replace("<s>", "").replace("</s>", "").strip()
    return q, a

# Parse
text_col = "text"
if text_col not in df_raw.columns:
    # Sometimes datasets use 'prompt' or similar; fallback: first column
    text_col = df_raw.columns[0]

parsed = df_raw[text_col].map(parse_inst)
df = pd.DataFrame(parsed.tolist(), columns=["question", "answer"])

# Keep raw for auditing if needed
df["raw_text"] = df_raw[text_col].astype(str).map(lambda x: x[:5000])  # cap length

# Drop unparsed
unparsed = df[df["question"].isna() | df["answer"].isna()].copy()
df = df.dropna(subset=["question", "answer"]).reset_index(drop=True)

# Make stable id
df["id"] = [sha1(f"{q}||{a}||{i}") for i, (q, a) in enumerate(zip(df["question"], df["answer"]))]

df = df[["id", "question", "answer", "raw_text"]]
df.head(), len(df), len(unparsed)


(                                         id  \
 0  cc79cd4e61194d7f30c078bd9f54aa0c94c43f64   
 1  f21051455d5c19c9b7b52096b4fafc3abc7bc88c   
 2  ca64cdd9d62e666f9c72debdbc24945302529c5a   
 3  1e348c10ee823d318858e51c7fddea63a0344ac2   
 4  bd311f898481fe58d118026962b1be27795adbc3   
 
                                             question  \
 0  สวัสดีค่ะ อยากรู้ว่าต้องทำยังไงคะมีอาการคันอวั...   
 1  อยากทราบว่ากินยาคลุมฉุกเฉินชนิด 1 เม็ด แล้วประ...   
 2  ไม่รู้ว่าใช่ประจำเดือนหรือป่าว แต่เลือกมันเป็น...   
 3  ตอนนี้หนูอายุ13ปีและหนูเป็นนางรำของร.ร.แต่หนูร...   
 4  ตามต้นหัวข้อเลยค่ะ เราเป็นคนที่คิดมากอยู่แล้ว ...   
 
                                               answer  \
 0  สวัสดีค่ะ อาการคันอวัยวะเพศหญิง อาจเกิดจาก-รูข...   
 1  สวัสดีค่ะ หลังใข้ยาคุมฉุกเฉินอาจทำให้มีเลือดออ...   
 2  สวัสดีค่ะ หากเลือดที่ออก อยู่ในช่วงวันที่ประจำ...   
 3  สวัสดีค่ะ ฟังจากเหตุการณ์และความคิดของ น่าจะมี...   
 4  สวัสดีค่ะ อาการร้องไห้ง่ายขึ้น มีพฤติกรรมทำร้า...   
 
                      

## 3) Privacy / PDPA + Safety filtering (recommended)

Even if this is a public dataset, the content may include:
- **minors** (age stated)
- sensitive sexual health topics
- mental health crises / self-harm mentions

We will:
1) Do a **conservative PII-risk scan** (email/phone/Thai ID format)  
2) Optionally filter **high-risk sensitive content** using keyword heuristics (editable list)

> This is NOT perfect. Treat it as an engineering safeguard + audit trail.


In [6]:
EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)
PHONE_RE = re.compile(r"\b(?:\+?\d{1,3}[- ]?)?(?:\(?\d{2,3}\)?[- ]?)?\d{3}[- ]?\d{4}\b")
THAI_ID_RE = re.compile(r"\b\d{1}-\d{4}-\d{5}-\d{2}-\d\b")

# Sensitive-topic heuristics (tune for your project)
SENSITIVE_PATTERNS = [
    r"ฆ่าตัวตาย", r"ทำร้ายตัวเอง", r"กรีด", r"ผูกคอ", r"อยากตาย",
    r"ข่มขืน", r"ล่วงละเมิด",
]
SENSITIVE_RE = re.compile("|".join(SENSITIVE_PATTERNS))

def pii_risk_flag(text: str) -> bool:
    if not text:
        return False
    if EMAIL_RE.search(text): return True
    if THAI_ID_RE.search(text): return True
    if PHONE_RE.search(text): return True
    return False

def sensitive_flag(text: str) -> bool:
    if not text:
        return False
    return bool(SENSITIVE_RE.search(text))

flags_pii = df.apply(lambda r: pii_risk_flag(r["question"]) or pii_risk_flag(r["answer"]), axis=1)
flags_sensitive = df.apply(lambda r: sensitive_flag(r["question"]) or sensitive_flag(r["answer"]), axis=1)

df_pii_flagged = df[flags_pii].copy()
df_sensitive_flagged = df[flags_sensitive].copy()

df_kept = df[~(flags_pii | flags_sensitive)].copy().reset_index(drop=True)

len(df), len(df_kept), len(df_pii_flagged), len(df_sensitive_flagged)


(189190, 186172, 358, 2667)

## 4) Dedupe + clean

- Dedupe by `(question, answer)`
- Drop empty rows


In [7]:
def dedupe(df_in: pd.DataFrame) -> pd.DataFrame:
    tmp = df_in.copy()
    tmp["_key"] = (tmp["question"].fillna("") + "||" + tmp["answer"].fillna("")).map(sha1)
    tmp = tmp.drop_duplicates(subset=["_key"]).drop(columns=["_key"]).reset_index(drop=True)
    return tmp

df_clean = dedupe(df_kept)
df_clean = df_clean[(df_clean["question"].str.len() > 0) & (df_clean["answer"].str.len() > 0)].reset_index(drop=True)

df_clean.shape


(186059, 4)

## 5) Split Train/Val/Test

This dataset may not have labels, so we do a random split by default.


In [8]:
def random_split(df_in: pd.DataFrame, train=0.8, val=0.1, test=0.1, seed=RANDOM_SEED):
    assert abs(train + val + test - 1.0) < 1e-9
    idx = list(df_in.index)
    rng = random.Random(seed)
    rng.shuffle(idx)
    n = len(idx)
    n_train = int(n * train)
    n_val = int(n * val)
    train_df = df_in.loc[idx[:n_train]].reset_index(drop=True)
    val_df = df_in.loc[idx[n_train:n_train+n_val]].reset_index(drop=True)
    test_df = df_in.loc[idx[n_train+n_val:]].reset_index(drop=True)
    return train_df, val_df, test_df

train_df, val_df, test_df = random_split(df_clean, 0.8, 0.1, 0.1)

len(train_df), len(val_df), len(test_df)


(148847, 18605, 18607)

## 6) Build retrieval corpus (chunk answers)

For a simple RAG baseline, we can index the **answers** (or combine question+answer).
If you later add external guideline PDFs, replace this with guideline chunking.


In [9]:
def chunk_text(text: str, max_chars: int = 900, overlap: int = 120):
    text = clean_text(text)
    if not text:
        return []
    # sentence-ish split (Thai doesn't use periods consistently; still helps a bit)
    sents = re.split(r"(?<=[.!?。！？])\s+", text)
    chunks, cur = [], ""
    for s in sents:
        if not s:
            continue
        if len(cur) + 1 + len(s) <= max_chars:
            cur = (cur + " " + s).strip()
        else:
            if cur:
                chunks.append(cur)
            cur = s
    if cur:
        chunks.append(cur)

    final = []
    for ch in chunks:
        if len(ch) <= max_chars:
            final.append(ch)
        else:
            start = 0
            while start < len(ch):
                final.append(ch[start:start + max_chars])
                start += max(1, max_chars - overlap)
    return final

def build_corpus(df_in: pd.DataFrame, max_chars=900):
    rows = []
    for _, r in df_in.iterrows():
        doc_id = r["id"]
        # Index answer alone, or (Q+A) if you prefer:
        base = f"Q: {r['question']}\nA: {r['answer']}"
        for j, ch in enumerate(chunk_text(base, max_chars=max_chars, overlap=max(20, int(max_chars*0.15)))):
            rows.append({"doc_id": doc_id, "chunk_id": f"{doc_id}-{j:03d}", "text": ch})
    return pd.DataFrame(rows)

corpus_df = build_corpus(train_df, max_chars=900)
corpus_df.head(), len(corpus_df)


(                                     doc_id  \
 0  cf4f80246e8f4aa4527acde329e708728ef2fb85   
 1  8a119092792fd342caa44cd757208b9b60b0cef2   
 2  8a119092792fd342caa44cd757208b9b60b0cef2   
 3  bf96be8699b1126bff81647bbe4bc236ae49e269   
 4  bf96be8699b1126bff81647bbe4bc236ae49e269   
 
                                        chunk_id  \
 0  cf4f80246e8f4aa4527acde329e708728ef2fb85-000   
 1  8a119092792fd342caa44cd757208b9b60b0cef2-000   
 2  8a119092792fd342caa44cd757208b9b60b0cef2-001   
 3  bf96be8699b1126bff81647bbe4bc236ae49e269-000   
 4  bf96be8699b1126bff81647bbe4bc236ae49e269-001   
 
                                                 text  
 0  Q: ผมโดนเเมวข่วนแมวยังไม่ฉีดยาพึ่งอายุ2เดือนกว...  
 1  Q: เป็นเวลาหนึ่งสัปดาห์แล้วที่ฉันรู้สึกแสบท้อง...  
 2  ราโซล 20 มก., แพนโทพราโซล 40 มก., โอเมพราโซล 2...  
 3  Q: คืิผมอานุแค่14ครับมีตุ่ทที่โคนลิ้นเจ็บไม่เจ...  
 4  เป็นอาการแสดงของโรคติดเชื้อต่างๆ เช่น โรคมือเท...  ,
 225332)

## 7) Export JSONL (+ audit files)

Outputs:
- `qa_train.jsonl`, `qa_val.jsonl`, `qa_test.jsonl`
- `corpus.jsonl`
- `pii_flagged.jsonl` (audit)
- `sensitive_flagged.jsonl` (audit)


In [10]:
def to_jsonl(df_in: pd.DataFrame, path: str):
    with open(path, "w", encoding="utf-8") as f:
        for row in df_in.to_dict(orient="records"):
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

outdir = "out_thai_med_pack"
os.makedirs(outdir, exist_ok=True)

to_jsonl(train_df.drop(columns=["raw_text"]), os.path.join(outdir, "qa_train.jsonl"))
to_jsonl(val_df.drop(columns=["raw_text"]),   os.path.join(outdir, "qa_val.jsonl"))
to_jsonl(test_df.drop(columns=["raw_text"]),  os.path.join(outdir, "qa_test.jsonl"))
to_jsonl(corpus_df, os.path.join(outdir, "corpus.jsonl"))

if len(df_pii_flagged) > 0:
    to_jsonl(df_pii_flagged, os.path.join(outdir, "pii_flagged.jsonl"))
if len(df_sensitive_flagged) > 0:
    to_jsonl(df_sensitive_flagged, os.path.join(outdir, "sensitive_flagged.jsonl"))

stats = {
    "rows_total_parsed": int(len(df)),
    "rows_unparsed": int(len(unparsed)),
    "rows_after_filters": int(len(df_clean)),
    "train": int(len(train_df)),
    "val": int(len(val_df)),
    "test": int(len(test_df)),
    "corpus_chunks_train": int(len(corpus_df)),
    "pii_flagged": int(len(df_pii_flagged)),
    "sensitive_flagged": int(len(df_sensitive_flagged)),
}
with open(os.path.join(outdir, "stats.json"), "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

stats


{'rows_total_parsed': 189190,
 'rows_unparsed': 0,
 'rows_after_filters': 186059,
 'train': 148847,
 'val': 18605,
 'test': 18607,
 'corpus_chunks_train': 225332,
 'pii_flagged': 358,
 'sensitive_flagged': 2667}

## 8) What to write in your proposal/report (copy-paste starter)

**Dataset & Data Description**
- Source: Hugging Face dataset `Thaweewat/thai-med-pack`
- Modality: Thai medical Q&A instruction-style text
- Size: ~189k rows (train split)
- Split: Created locally (80/10/10)

**Privacy / PDPA / Ethics**
- Content may include sensitive health topics and minors; treat as potentially sensitive.
- Mitigations: PII heuristic scan + sensitive-topic filter + audit outputs.
- Risk if wrong: incorrect health guidance -> require disclaimers, escalation rules, and “decision-support only”.

**Data Engineering**
- Parse instruction format, dedupe, drop unparsed/empty.
- Sampling: random split (no labels).
- Assumptions: parsed Q/A reflect intended supervision signal; filtering reduces harmful/sensitive content exposure.
